# SPECFEM Users Workshop -- Day 2 (Oct. 6, 2022)
## Day 2b: Kernel Exercise


- In this notebook we will have participants run their own adjoint simulation using Day 2a as a guide 
- We will use two homogeneous halfspace models for simplicity, building on the exercise from Day 1b  
- Adjoint simulations are key for performing seismic imaging (Day 3) as their results guide iterative model updates during the inverse problem  
- These instructions should be run from inside the Docker container, using Jupyter Lab (see *Docker Preamble* in Day 0). 

-----------

**Relevant Links:** 
- Today's Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_2b_kernels_exercise.ipynb
- Completed Notebook https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_2b_kernels_exercise.ipynb
- Day 0 Notebook (Container Testing): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_0_container_testing.ipynb
- Day 1A Notebook (Intro SPECFEM): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1a_intro_specfem2d.ipynb
- Day 1B Notebook (Fwd. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1b_forward_simulations.ipynb
- Day 2A Notebook (Adj. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_2a_kernels.ipynb

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.

-----------
## 0) Setting Up a SPECFEM2D Working Directory

- Let's set up a clean working directory to run SPECFEM2D  
- We will be doing all our work in the directory `/home/scoped/work/day_2/exercise`, all the following cells assume that we are in this directory  

In [ ]:
# Python packages we might use in this notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
# Make correct dir. and move there
! mkdir -p /home/scoped/work/day_2/exercise
%cd /home/scoped/work/day_2/exercise

# Symlink the executables and copy the relevant DATA/ directory
! ln -s /home/scoped/specfem2d/bin .
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
! cp -f DATA/Par_file_Tape2007_onerec DATA/Par_file

# Ensure that SPECFEM outputs required files for adjoint simulations
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1

! mkdir OUTPUT_FILES

! ls

------------
## 1) Target Model Forward Simulations

- We'll use the homogeneous halfspace defined by default in the example as our target model
- Remember that the model spans X = [0, 480000]m and Z = [0, 480000]m  
- Also remember that there are 40 elements in X and Z, corresponding to element sizes of 12000m  
- We need to set up the working directory before running our simulation

### 1a) STATIONS
- Using what you learned in Day 1, generate your own STATIONS file with an interesting configuration
- Look at *'STATIONS_checker'* to get a refresher of how the STATIONS file should be configured
- Try to use 100 stations to design an array configuration of your choice  
- Potential configurations you could choose that might mimic a real world seismic array:  
    a) Cross shaped linear array  
    b) Uniform, dense gridded array  
    c) Spiral  
    d) Concentric rings  
    e) Dense linear array mimicing a DAS sensor  
    


### 1b) SOURCE
- Create your own source with custom coordinates and moment tensor components  
- Use one of the available 'SOURCE_???' files as a template
- *Remember* that SPECFEM2D is expecting a file called SOURCE  
- Can you think of an interesting moment tensor configuration? Explosions are always fun!  
- Can you place the SOURCE somewhere it would be interesting based on your station configuration? e.g.,  
    - at the center of a spiral or concentric rings  
    - normal to a linear array or grid  
    - at one end of a linear array

### 1c) Run the Mesher and Solver  

1) *Remember* to tell SPECFEM to use your STATIONS file, and not it's internal definition of stations
2) Run your simulation in **parallel** using 4 cores
    - *Remember* that you need to tell both SPECFEM and MPI that you are planning to run 4 processes  

The remainder of the `Par_file` should already be set up appropriately

### 1d) Save Your Results
- Make sure you **save the seismograms** output by SPECFEM somewhere safe  
- Subsequent simulations will **overwrite** files in the DATA/ and OUTPUT_FILES/ directory  
- Remember that displacement seismograms are stored in the *OUTPUT_FILES/* directory with the extension '*.semd*'

------------
## 2) Initial Model Forward Simulations

- Let's edit the current model definition to create a separate initial or 'starting' model
- The starting model will also be a homogeneous halfspace, but with slightly different velocities 
- We'll use what we learned in the [Day 1B exercise](https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1b_forward_simulations.ipynb) to change the model parameters  

### 2a) Edit Velocity Model
- *Remember* that the velocity model is defined in the `Par_file`
- Let's **decrease** the velocity values (Vp and Vs) of the starting model by 20\%  
- In other words $V_p \rightarrow V_p - V_p \times 0.2$
- *Remember* from Day 1B that we can use SeisFlows to view and change the velocity model parameters  

### 2b) Set the `Par_file` for a Forward Simulation

1) Tell SPECFEM to **save the forward wavefield** after the simulation  
2) Tell SPECFEM to **output binary database files** (as opposed to ASCII files)  
3) Tell SPECFEM to **save the model** in binary format

### 2c) Run the Mesher, Solver and Save Results  

1) Run your simulation in **parallel** using 4 cores
2) After your simulation, **save the seismograms** output by SPECFEM somewhere safe  

### 2d) Optional: Visualize Waveforms

- To make sure the two models were different, we can check our waveforms against one another  
- Use Python to plot matching seismograms against one another  
- Alternatively, you can use RecSec to plot *both* sets of synthetics, you'll need the following flags 
    - `--pysep_path`: path to tell RecSec where your 'data' is  
    - `--syn_path`: path to tell RecSec where your 'synthetics' are  
    - `--cmtsolution`: path to your 'SOURCE' file  
    - `--stations`: path to your 'STATIONS' file  
    - `--components`: the components of your seismograms. These are listed in the filenames (e.g., AA.S000099.BXY.semd is component 'Y')  
    - `--synsyn`: flag to tell RecSec that we are plotting two sets of synthetics (not actual data)  
    - `--cartesian`: flag to tell RecSec that our domain is cartesian (not geographic) 

---------------
## 3) Quantify Misfit, Generate Adjoint Sources

- You should now have **two sets of synthetics**, one generated by your initial model, another by your target model
- We now want to generate adjoint sources for each pair of synthetics
- Let's use a cross correlation traveltime misfit function
- The cross correlation misfit is defined: $\chi (\mathbf{m}) = \frac{1}{2} \left[ T^{obs} - T(\mathbf{m}) \right] ^ 2$,  
- Where $T^{obs}$ is the observed traveltime, and $T(\mathbf{m})$ is the
predicted traveltime in Earth model $m$  
- **Alternatively**, you can use the waveform difference objective function we say in Day 2 Section 3A  

### 3a) Define your Misfit Function

>__Adjoint Source Equation:__ $f^{\dagger}(t) = - \left[ T^{obs} - T(\mathbf{m}) \right] ~ \frac{1}{N} ~
    \partial_t \mathbf{s}(T - t, \mathbf{m})$
       
Complete the function below using the following steps:

1) Calculate the time shift $\left[ T^{obs} - T(\mathbf{m})\right]$ using [ObsPy's cross correlation function](https://docs.obspy.org/master/packages/autogen/obspy.signal.cross_correlation.correlate.html) 
    - The correlate function returns an array of correlation values
    - Use [xcorr_max](https://docs.obspy.org/master/packages/autogen/obspy.signal.cross_correlation.xcorr_max.html#obspy.signal.cross_correlation.xcorr_max) to find the time shift related to the peak cross correlation
    - The time shift should be a **single value**
2) Differentiate the synthetic waveform, $\partial_t \mathbf{s}(t, \mathbf{m})$,  using [NumPy gradient](https://numpy.org/doc/stable/reference/generated/numpy.gradient.html)  
3) Set the normalization factor $N$ as:  $N = \int_0^T ~ \mathbf{s}(t, \mathbf{m}) ~ \partial^2_t \mathbf{s}(t, \mathbf{m}) dt$  
    - Where T is the total seismogram time
    - Use [SciPy's Simpson's rule](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.simps.html) to integrate  
4) Time reverse the differentiated synthetic waveform $\partial_t \mathbf{s}(T - t, \mathbf{m})$  
5) Return an adjoint source that combines parts 1 through 4 that follows the **Adjoint Source Equation** above. Remember the -1!


---------
Start with the following template and try to follow steps above:
```python
from numpy import gradient
from scipy.integrate import simps
from obspy.signal.cross_correlation import correlate, xcorr_max


def cc_traveltime_adjsrc(d, s):
    """
    Define a cross-correlation traveltime adjoint source
    
    :type d: np.array
    :param d: data array
    :type s: np.array
    :param s: synthetic array
    :rtype adj_src: np.array
    :return adj_src: adjoint source array
    """
    # follow steps 1-5 here
    # ...
    return adj_src
```

### 3b) Generate Adjoint Sources

1) **Loop** through all available data and synthetic seismogram files, make sure filenames match!  
1) **Load** in data and synthetic seismogram for a single station (use `numpy.loadtxt`; see Day 2A; Section 2C)
2) **Apply** your adjoint source function from 3A to **output** an adjoint source array  
3) **Save** the corresponding adjoint source in the `SEM/` directory (using `numpy.savetxt`; see Day 2A; Section 3A)
    - *Remember* to format the adjoint source the same as the input synthetics
    - *Remember* that adjoint sources must mimic the synthetic filename, but end with *.adj*
    
*Feel free to import Python modules required to file match and loop!*  

## 3c) Optional: Plot an adjoint source

- It's always useful to look at adjoint sources before running an adjoint simulation  
- Plot **one** adjoint source next to its corresponding data and synthetic waveforms if you can  
- We should be able to immediately tell if the adjoint source looks appropriate  
- *Remember* that we plotted adjoint sources in Day 2a Section 3b

------------
## 4) Run Adjoint Simulations

- *Remember* to tell SPECFEM that this is an adjoint simulation (not a forward simulation)
- *Remember* to tell SPECFEM to **output** kernel files in FORTRAN Binary format
- Make sure that your adjoint sources are stored in the `SEM/` directory (Step 3)  
- Make sure your DATABASE files are available in the *OUTPUT_FILES/* directory (Step 2)  
- Remember you do **not** need to run the mesher, only the solver  
- Check the output log file and kernel files to make sure you ran an adjoint simulation (not forward)

## 5) Smooth Kernel
- **Smooth** your Vp and Vs kernels by 10km in the horizontal direction and 10 km in the vertical
- Make sure that SPECFEM can find the appropriate files (kernels, database and model files are all locatable in the same directory)  
- Look at Day 2A Section 4 if you need help calling the smoothing executable  
- Remember to run the smoother with 4 MPI processors  

-------------
## 6) Visualize Results
- Use SeisFlows (see Day 2a, Section 4) or NumPy + Matplotlib to visualize your kernel results  
- Does your kernel make sense?
- Can you plot the sources and stations on top of the kernel figure?
- **NOTE:** If you use SeisFlows, you'll need to import the Model tool and change the names of the kernels from 'alpha' -> 'vp' and 'beta' -> 'vs'

```python 
from seisflows.tools.specfem import Model
# OR
from seisflows.tools.specfem import read_fortran_binary
```